In [2]:
from pyspark import SparkContext, SparkConf
import matplotlib as plt
import seaborn as sb

conf = SparkConf().setAppName('finger-2').setMaster('local')
sc = SparkContext(conf=conf)

# Punto 1. Buscamos el evento predominante

In [47]:
rdd = sc.textFile('../events.csv')
resultados = rdd.map(lambda s: s.split(',')).map(lambda s: (s[1], 1)).reduceByKey(lambda a, b: a + b).collect()

## Considero que la cantidad de información traida es lo suficientemente pequeña como para poder ser procesada en el cliente. Siendo que 'event' es un categórico y no debería crecer proporcionalmente a la cantidad de datos, no me parece incorrecto procesarlo con el cliente.
## Sin embargo, lo encararé de ambas formas
resultados.sort(key=lambda tup:tup[1], reverse = True)

print('Procesamiento del lado del cliente:')
print('El evento predominante es << {} >> con {} apariciones'.format(resultados[0][0], resultados[0][1]))
print(' ')

resultados = rdd.map(lambda s: s.split(',')).map(lambda s: (s[1], 1)).reduceByKey(lambda a, b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b)

print('Procesamiento del lado del servidor:')
print('El evento predominante es << {} >> con {} apariciones'.format(resultados[0], resultados[1]))


Procesamiento del lado del cliente:
El evento predominante es << viewed product >> con 528931 apariciones
 
Procesamiento del lado del cliente:
El evento predominante es << viewed product >> con 528931 apariciones


# Punto 2. Producto más visitado

In [56]:
productos = rdd.map(lambda x: x.split(',')).filter(lambda x: x[1] == 'viewed product').map(lambda x: (x[5], 1)).reduceByKey(lambda a, b: a + b)

print('Los productos más visitados son:')
productos.takeOrdered(5, key = lambda x : -x[1])

Los productos más visitados son:


[('iPhone 6', 47538),
 ('iPhone 5s', 42045),
 ('iPhone 6S', 36930),
 ('iPhone 7', 26330),
 ('Samsung Galaxy S7 Edge', 23062)]